In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import seaborn as sns
import plotnine as gg
from mpl_toolkits.mplot3d import Axes3D
from pylab import *
from sklearn.manifold import TSNE
from palettable.colorbrewer.sequential import *
from pathlib import Path
import umap.umap_ as umap
from scipy import stats
from scipy.stats import ttest_ind


In [194]:
top_dir = os.path.dirname(os.getcwd())
#outpath = os.path.join(top_dir, "Figures")
outpath = "/Users/habbasi/Documents/Github/broadinstitute/ProteinKinase_NathianelGray/Figures/Features_map/ERK5/"

## Data Paths

In [219]:
path = Path("../collated/")
#outpath = Path("/Welch_ttest/")
fname = "C-7210-01-CMP-008-gray_collapsed_sampled.csv"
fpath = os.path.join(path, fname)
df = pd.read_csv(fpath)


metadata = [col for col in df.columns if "Metadata" in col]
var = [col for col in df.columns if not "Metadata" in col]
exclude_var = ['Cells_Correlation_Costes_DNA_Mito',
 'Cytoplasm_Correlation_Costes_DNA_Mito',
 'Cytoplasm_Correlation_Costes_DNA_RNA']

variables = [v for v in var if v not in exclude_var]

df['Metadata_Target'].replace(np.nan, 'NA', inplace=True)
df['Metadata_Annotation'].replace('active ', 'active', inplace=True)


c=['A549']
p = ['SECRET']
excluded = ["C1", "C2", "C3", "C4"]

compd = ['BSJ-04-030', 'BSJ-03-136']

d = df.query('Metadata_cell_line in @ c and Metadata_compound_name not in @ excluded and Metadata_Target in @p')

d.Metadata_compound_name.unique()

dp = d.query("Metadata_compound_name == 'BSJ-03-136'")

#dp.Metadata_mmoles_per_liter.unique()
dp.head()

,Metadata_Plate,Metadata_Well,Metadata_broad_sample,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_cell_line,Metadata_well_position,Metadata_mg_per_ml,Metadata_mmoles_per_liter,Metadata_solvent,...,Nuclei_Texture_SumAverage_Mito_5_0,Nuclei_Texture_SumAverage_RNA_20_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_DNA_20_0,Nuclei_Texture_SumEntropy_DNA_5_0,Nuclei_Texture_SumEntropy_RNA_5_0,Nuclei_Texture_SumVariance_DNA_20_0,Nuclei_Texture_Variance_ER_20_0,Nuclei_Texture_Variance_Mito_20_0,Nuclei_Texture_Variance_RNA_10_0
172,BR00100032,H05,BRD-U43181200-000-01-8,BR00100032,C-7210-01-CMP-008-gray,A549,H05,NaN,2.5,DMSO,...,0.692729,-0.107442,0.326947,1.536125,-1.034746,0.361871,3.301785,5.610193,0.782833,0.069363
176,BR00100032,H09,BRD-U43181200-000-01-8,BR00100032,C-7210-01-CMP-008-gray,A549,H09,NaN,2.5,DMSO,...,0.404829,-0.660995,0.834810,2.136472,-0.306462,-0.272973,4.227064,6.174764,0.812594,-0.764022
185,BR00100032,H18,BRD-U43181200-000-01-8,BR00100032,C-7210-01-CMP-008-gray,A549,H18,NaN,2.5,DMSO,...,1.446802,0.532957,0.334157,2.072267,-1.052212,1.368904,3.715210,8.134602,1.461905,0.379571
189,BR00100032,H22,BRD-U43181200-000-01-8,BR00100032,C-7210-01-CMP-008-gray,A549,H22,NaN,2.5,DMSO,...,2.345343,1.898227,1.729718,2.728664,0.209631,1.773838,5.744086,5.112244,1.352662,0.711656
194,BR00100032,I03,BRD-U43181200-000-01-8,BR00100032,C-7210-01-CMP-008-gray,A549,I03,NaN,2.5,DMSO,...,0.676421,-0.174502,-0.275737,0.827803,-1.387262,0.606489,3.246210,6.675981,0.938030,-0.186807


## Welch Ttest

In [190]:
c=['A549']
p = ['ERK5']
excluded = ["C1", "C2", "C3", "C4"]


features_group=['RadialDistribution']
channels=['Mito', 'RNA']



d = df.query('Metadata_cell_line in @ c and Metadata_compound_name not in @ excluded and Metadata_Target in @p')

varlist = (d.columns[d.columns.str.contains(features_group[0])]
           .tolist()
          )


varlist = [match for match in varlist if channels[1] in match]

varlist 


tt = []

for i in varlist:
    test = d.Metadata_compound_name.unique().tolist()
    
    gp1 = d.query("Metadata_compound_name in @ test[1]")[i].values
    gp2 = d.query("Metadata_compound_name in @ test[0]")[i].values

 

    t1 = ttest_ind(gp1, gp2, equal_var=False)




    d1 = {"Paired": [test[0]+', '+ test[1]],
          "Statistics": [t1[0]],
          "P-value": [t1[1]],
          "Variables": str(i)}



    prf = pd.DataFrame(d1, columns = ['Paired','Statistics', 'P-value', 'Variables'])

    tt.append(prf)
final_tt = pd.concat(tt)


# converting P-values to -log10[pvalues]
final_tt['logp'] = -log10(final_tt['P-value'])
                    
#index = [i for i in range(1, len(final_tt) +1)]
#final_tt['index'] =  index
 
final_tt = (final_tt
            .sort_values(by='logp', ascending=False))


#final_tt['Variables'].tolist()[0:5]
final_tt.head(5)

# for i, var in enumerate(varlist):
#     g = gg.ggplot() + \
#     gg.geom_density(gg.aes(x=str(var), y='stat(density)', color = 'Metadata_compound_name', fill= 'Metadata_compound_name'), data= d, alpha=.1) + \
#     gg.xlab("Feature_Name") + \
#     gg.ylab("Density") + \
#     gg.labs(title=str(var)) + \
#     gg.theme_classic() 



#     gg.ggsave(filename= str(var)+ ".png", plot = g, path = outpath)




,Paired,Statistics,P-value,Variables,logp
0,"FMF-04-159-2, FMF-05-176-1",-52.279437,1.294305e-16,Cytoplasm_RadialDistribution_RadialCV_RNA_4of4,15.887963
0,"FMF-04-159-2, FMF-05-176-1",-38.991059,4.433907e-15,Cytoplasm_RadialDistribution_RadialCV_RNA_3of4,14.353213
0,"FMF-04-159-2, FMF-05-176-1",-50.234685,1.764453e-14,Cells_RadialDistribution_RadialCV_RNA_4of4,13.753390
0,"FMF-04-159-2, FMF-05-176-1",-26.947552,3.962486e-13,Cytoplasm_RadialDistribution_MeanFrac_RNA_1of4,12.402032
0,"FMF-04-159-2, FMF-05-176-1",-25.462702,5.124997e-13,Cells_RadialDistribution_RadialCV_RNA_3of4,12.290306


## Neighbors Features

In [201]:
c=['A549']
p = ['CDK14 / pan-TAIRE']
excluded = ["C1", "C2", "C3", "C4"]


features_group='AreaShape'



d = df.query('Metadata_cell_line in @ c and Metadata_compound_name not in @ excluded and Metadata_Target in @p')

varlist = (d.columns[d.columns.str.contains(features_group)]
           .tolist()
          )

varlist = [match for match in varlist if "Nuclei" in match]



tt = []

for i in varlist:
    test = d.Metadata_compound_name.unique().tolist()
    
    gp1 = d.query("Metadata_compound_name in @ test[1]")[i].values
    gp2 = d.query("Metadata_compound_name in @ test[0]")[i].values

 

    t1 = ttest_ind(gp1, gp2, equal_var=False)




    d1 = {"Paired": [test[0]+', '+ test[1]],
          "Statistics": [t1[0]],
          "P-value": [t1[1]],
          "Variables": str(i)}



    prf = pd.DataFrame(d1, columns = ['Paired','Statistics', 'P-value', 'Variables'])

    tt.append(prf)
final_tt = pd.concat(tt)


# converting P-values to -log10[pvalues]

final_tt['logp'] = -log10(final_tt['P-value'])
 
final_tt = (final_tt
            .sort_values(by='logp', ascending=False))


final_tt.head(6)
final_tt['Variables'].tolist()[0:7]

#index = [i for i in range(1, len(final_tt) +1)]
#final_tt['index'] =  index
 
 

# for i, var in enumerate(varlist):
#     g = gg.ggplot() + \
#     gg.geom_density(gg.aes(x=str(var), y='stat(density)', color = 'Metadata_compound_name', fill= 'Metadata_compound_name'), data= d, alpha=.1) + \
#     gg.xlab("Feature_Name") + \
#     gg.ylab("Density") + \
#     gg.labs(title=str(var)) + \
#     gg.theme_classic() 



#     gg.ggsave(filename= str(var)+ ".png", plot = g, path = outpath)



['Nuclei_AreaShape_Zernike_2_2',
 'Nuclei_AreaShape_Zernike_5_1',
 'Nuclei_AreaShape_Eccentricity',
 'Nuclei_AreaShape_Zernike_6_4',
 'Nuclei_AreaShape_Zernike_8_4',
 'Nuclei_AreaShape_Zernike_0_0',
 'Nuclei_AreaShape_Compactness']

In [155]:
c=['A549']
p = ['NA','ERK5']
excluded = ["C1", "C2", "C3", "C4"]

d = df.query('Metadata_cell_line in @ c and Metadata_compound_name not in @ excluded and Metadata_Target in @p')


d.Metadata_Target.unique()

array(['NA', 'ERK5'], dtype=object)

In [161]:
d.Metadata_compound_name.unique()

array(['DMSO', 'JWG-071', 'AX15836', 'JWG-119'], dtype=object)